In [1]:
import re
import os
import pandas as pd
import numpy as np
from project import configs
from GSEpipelineFast import *


In [2]:
filename = 'Disease_Gene_Analyzed.xlsx'
inqueryFullPath = os.path.join(configs.rootdir, 'data', filename)
xl = pd.ExcelFile(inqueryFullPath)
sheet_name = xl.sheet_names
inqueries = pd.read_excel(inqueryFullPath, sheet_name=sheet_name, header=0)


In [3]:
inqueries['Sheet1']

,GSE ID,Samples,Source,Experiment,Time,GPL ID,Instrument
0,GSE21942,GSM1066005,NaN,control,NaN,GPL570,affy
1,NaN,GSM1066006,NaN,control,NaN,NaN,NaN
2,NaN,GSM1066007,NaN,control,NaN,NaN,NaN
3,NaN,GSM1066008,NaN,control,NaN,NaN,NaN
4,NaN,GSM1066009,NaN,control,NaN,NaN,NaN
5,NaN,GSM1066010,NaN,control,NaN,NaN,NaN
6,NaN,GSM1066011,NaN,control,NaN,NaN,NaN
7,NaN,GSM1066012,NaN,control,NaN,NaN,NaN
8,NaN,GSM1066013,NaN,control,NaN,NaN,NaN
9,NaN,GSM1066014,NaN,control,NaN,NaN,NaN


In [14]:
inqueries['Sheet1'].fillna(method='ffill',inplace=True)
df = inqueries['Sheet1'].loc[:,['GSE ID','Samples','GPL ID','Instrument']]
df_target = inqueries['Sheet1'].loc[:,['Samples','Experiment']]
df_target.rename(columns={'Samples':'FileName','Experiment':'Condition'},inplace=True)
df_target['FileName'] = df_target['FileName'].astype(str) + '.txt.gz'
df_target['SampleNumber']= 1 + df_target.index.values
df_target = df_target[['SampleNumber','FileName','Condition']]

In [5]:
def queryTest(df):
    sr = df['GSE ID']
    gse_ids = sr[sr.str.match('GSE')].unique()
    sr = df['Samples'].dropna()
    gsm_ids = sr.unique()
    print('---\nStart Collecting Data for:')
    print(gse_ids)
    print(gsm_ids)
    print('---\n')
    # fetch data of each gse if it is not in the database, update database
    for gse_id in gse_ids:
        querytable = df[df['GSE ID']==gse_id]
        gseXXX = GSEproject(gse_id,querytable,configs.rootdir)
        if lookupTranscriptomicsDB(gseXXX):
            print("{} already in database, skip over.".format(gseXXX.gsename))
            continue
        updateTranscriptomicsDB(gseXXX)

    df_results = fetchLogicalTable(gsm_ids)
    df_output = mergeLogicalTable(df_results)

    posratio = df_output.sum(axis=1,skipna=True)/df_output.count(axis=1)
    df_output['Pos'] = posratio
    df_output['0.5'] = np.where(posratio >= 0.5, 1, 0)
    df_output['0.9'] = np.where(posratio >= 0.9, 1, 0)

    return df_output


In [22]:
querytable = df
targetdir = os.path.join(configs.datadir,'targets.txt')
df_target.to_csv(targetdir, index=False, sep='\t')
df_target

,SampleNumber,FileName,Condition
0,1,GSM1066005.txt.gz,control
1,2,GSM1066006.txt.gz,control
2,3,GSM1066007.txt.gz,control
3,4,GSM1066008.txt.gz,control
4,5,GSM1066009.txt.gz,control
5,6,GSM1066010.txt.gz,control
6,7,GSM1066011.txt.gz,control
7,8,GSM1066012.txt.gz,control
8,9,GSM1066013.txt.gz,control
9,10,GSM1066014.txt.gz,control


In [18]:
gseXXX = GSEproject('GSE21942',querytable,configs.rootdir)

Initialize project (GSE21942):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066005.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066006.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066007.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066008.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066009.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066010.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066011.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066012.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066013.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/

In [9]:
gseXXX.gsm_platform

{'GSM1066005': 'GPL570',
 'GSM1066006': 'GPL570',
 'GSM1066007': 'GPL570',
 'GSM1066008': 'GPL570',
 'GSM1066009': 'GPL570',
 'GSM1066010': 'GPL570',
 'GSM1066011': 'GPL570',
 'GSM1066012': 'GPL570',
 'GSM1066013': 'GPL570',
 'GSM1066014': 'GPL570',
 'GSM1066015': 'GPL570',
 'GSM1066016': 'GPL570',
 'GSM1066017': 'GPL570',
 'GSM1066018': 'GPL570',
 'GSM1066019': 'GPL570',
 'GSM1066020': 'GPL570',
 'GSM1066021': 'GPL570',
 'GSM1066022': 'GPL570',
 'GSM1066023': 'GPL570',
 'GSM1066024': 'GPL570'}

In [10]:
df_gse_data = gseXXX.get_entrez_table_pipeline()
df_gse_data

Create new table: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSE21942_sc500_full_table.csv
affy Read Path: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GPL570
background correction: mas 
PM/MM correction : mas 
expression values: mas 
background correcting...

R[write to console]: 



done.
54675 ids to be processed
|                    |
|####################|
Getting probe level data...
Computing p-values
Making P/M/A Calls
GPL570: (44134, 60)
Full: (44134, 60)
Full table saved to:
/Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSE21942_sc500_full_table.csv


,gsm1066012.cel.gz,gsm1066012.cel.gz.1,gsm1066012.cel.gz.2,gsm1066017.cel.gz,gsm1066017.cel.gz.1,gsm1066017.cel.gz.2,gsm1066024.cel.gz,gsm1066024.cel.gz.1,gsm1066024.cel.gz.2,gsm1066011.cel.gz,...,gsm1066016.cel.gz.2,gsm1066010.cel.gz,gsm1066010.cel.gz.1,gsm1066010.cel.gz.2,gsm1066018.cel.gz,gsm1066018.cel.gz.1,gsm1066018.cel.gz.2,gsm1066006.cel.gz,gsm1066006.cel.gz.1,gsm1066006.cel.gz.2
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,221.586252,A,0.091157,233.281260,A,0.065333,240.578891,A,0.123873,254.271826,...,0.266847,345.487768,A,0.077430,192.933482,A,0.106612,189.772677,A,0.164032
10,17.735941,A,0.954420,15.651106,A,0.934667,30.919125,A,0.908843,43.758208,...,0.733153,8.529239,A,0.986884,16.655444,A,0.979570,18.000946,A,0.979570
100,2108.927293,P,0.002220,1687.302238,P,0.001673,1700.515755,P,0.004963,1997.287524,...,0.003823,855.925509,P,0.010398,1511.265518,P,0.003823,1444.894653,P,0.002220
1000,24.428486,A,0.893388,15.087666,A,0.703144,205.898267,A,0.296856,43.746642,...,0.211798,25.095375,A,0.535424,42.719692,A,0.761547,166.473473,A,0.266847
10000,1094.008597,P,0.001673,929.175458,P,0.003823,1302.679609,P,0.001673,1310.944525,...,0.002220,1333.178998,P,0.001673,858.391458,P,0.002220,1625.419707,P,0.001673
100009676,169.856485,A,0.143002,92.959285,A,0.361054,59.602566,A,0.429432,83.495643,...,0.328321,128.924614,A,0.500000,122.922792,A,0.570568,122.643554,A,0.394838
10001,1455.832736,P,0.004963,1561.128763,P,0.002220,1137.248596,P,0.001673,1359.687729,...,0.002220,1622.719058,P,0.002924,1187.900616,P,0.003823,1067.913643,P,0.001673
10002,32.409733,A,0.361054,72.806517,A,0.123873,66.013895,A,0.266847,15.126221,...,0.429432,57.862465,A,0.394838,149.506576,A,0.077430,92.291889,A,0.238453
10003,89.812613,M,0.054755,45.659226,A,0.091157,144.372064,P,0.030942,62.364951,...,0.238453,95.194824,A,0.143002,66.849572,A,0.164032,55.287722,A,0.429432


In [ ]:
rawdir = 
data2 = affyio.fitaffydir("/Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GPL570/")

background correction: mas 
PM/MM correction : mas 
expression values: mas 
background correcting...done.
54675 ids to be processed
|                    |
|####

In [19]:
gseXXX.genedir

'/Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW'

In [25]:
for key,val in gseXXX.platforms.items():
    rawdir = os.path.join(gseXXX.genedir,key)
    print('{}:{}, {}'.format(key, val, rawdir))
    data2 = affyio.fitaffydir(rawdir, targetdir)

GPL570:affy, /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GPL570
Background correcting
Normalizing
Calculating Expression


In [26]:
data2

,logFC,AveExpr,t,P.Value,adj.P.Val,B
244674_at,-1.363539,7.056140,-8.971584,9.388161e-09,0.000320,9.837268
200699_at,-0.985601,7.752403,-8.852896,1.181152e-08,0.000320,9.637606
224962_at,-0.833903,7.701195,-8.558819,2.102852e-08,0.000320,9.133441
217844_at,-0.507531,9.308057,-8.504085,2.344090e-08,0.000320,9.038106
232499_at,-0.770630,6.223639,-8.010608,6.352988e-08,0.000695,8.157129
227937_at,-0.491230,7.206133,-7.918674,7.677170e-08,0.000700,7.988716
218522_s_at,-0.730068,6.945137,-7.721186,1.157420e-07,0.000870,7.622366
228340_at,-0.656001,6.918263,-7.575609,1.571703e-07,0.000870,7.348319
202617_s_at,-0.674560,7.202826,-7.561699,1.618571e-07,0.000870,7.321957
225530_at,-0.866626,9.677202,-7.532126,1.723058e-07,0.000870,7.265807


In [32]:
data2['abs_logFC'] = data2['logFC'].abs()
data2.sort_values(by='abs_logFC', ascending=False, inplace=True)
regulated = data2[data2['abs_logFC']>=1.0]
down_regulated = regulated[regulated['logFC']<0]
up_regulated = regulated[regulated['logFC']>0]

In [33]:
down_regulated

,logFC,AveExpr,t,P.Value,adj.P.Val,B,abs_logFC
210387_at,-1.929387,5.455615,-3.417228,2.502000e-03,0.066673,-1.564960,1.929387
205898_at,-1.883483,9.534293,-3.375854,2.760329e-03,0.070689,-1.655318,1.883483
1564139_at,-1.796157,8.140986,-4.134635,4.443702e-04,0.025548,0.034517,1.796157
235811_at,-1.732083,5.674150,-5.905458,6.409976e-06,0.003494,3.969937,1.732083
212070_at,-1.664695,9.793336,-3.648518,1.439278e-03,0.048580,-1.055027,1.664695
215894_at,-1.511393,6.301930,-6.649406,1.176139e-06,0.001942,5.526368,1.511393
205495_s_at,-1.508454,10.636334,-3.508565,2.012585e-03,0.058962,-1.364499,1.508454
230464_at,-1.465048,8.515121,-3.950487,6.945053e-04,0.033009,-0.380139,1.465048
37145_at,-1.393113,10.419896,-3.414683,2.517183e-03,0.066939,-1.570526,1.393113
210321_at,-1.386158,10.634978,-2.748618,1.181053e-02,0.155974,-2.978555,1.386158


In [34]:
up_regulated

,logFC,AveExpr,t,P.Value,adj.P.Val,B,abs_logFC
204439_at,2.158360,7.054134,3.297925,0.003319,0.078486,-1.824685,2.158360
202388_at,1.766650,8.585433,4.856222,0.000077,0.010803,1.661930,1.766650
202917_s_at,1.751238,7.295925,3.071948,0.005636,0.104817,-2.308774,1.751238
204959_at,1.688234,5.611217,3.311006,0.003218,0.076996,-1.796335,1.688234
202859_x_at,1.604780,6.843914,2.494706,0.020720,0.210215,-3.480346,1.604780
202973_x_at,1.571663,7.403992,3.353213,0.002913,0.072714,-1.704641,1.571663
229450_at,1.530566,5.611778,2.062749,0.051301,0.331179,-4.270299,1.530566
221731_x_at,1.510117,4.178569,4.390985,0.000238,0.018624,0.613392,1.510117
201739_at,1.509796,6.747399,5.811887,0.000008,0.003856,3.768881,1.509796
226702_at,1.500821,6.909738,2.844167,0.009515,0.138290,-2.783859,1.500821


In [46]:
dataAll = fetch_entrez_gene_id(list(data2.index.values), input_db='Affy ID')
dataAll.replace(to_replace='-', value=np.nan, inplace=True)
data2.index.name='Affy ID'
data2['ENTREZ_GENE_ID'] = dataAll['Gene ID']
data2.dropna(how='any',subset=['ENTREZ_GENE_ID'],inplace=True)
data2.to_csv(os.path.join(configs.datadir,'Fit_GSE21962.csv'), index=False)

INFO    [bioservices:BioDBNet]:  Initialising BioDBNet service (REST)


retrieve 0:500
retrieve 500:1000
retrieve 1000:1500
retrieve 1500:2000
retrieve 2000:2500
retrieve 2500:3000
retrieve 3000:3500
retrieve 3500:4000
retrieve 4000:4500
retrieve 4500:5000
retrieve 5000:5500
retrieve 5500:6000
retrieve 6000:6500
retrieve 6500:7000
retrieve 7000:7500
retrieve 7500:8000
retrieve 8000:8500
retrieve 8500:9000
retrieve 9000:9500
retrieve 9500:10000
retrieve 10000:10500
retrieve 10500:11000
retrieve 11000:11500
retrieve 11500:12000
retrieve 12000:12500
retrieve 12500:13000
retrieve 13000:13500
retrieve 13500:14000
retrieve 14000:14500
retrieve 14500:15000
retrieve 15000:15500
retrieve 15500:16000
retrieve 16000:16500
retrieve 16500:17000
retrieve 17000:17500
retrieve 17500:18000
retrieve 18000:18500
retrieve 18500:19000
retrieve 19000:19500
retrieve 19500:20000
retrieve 20000:20500
retrieve 20500:21000
retrieve 21000:21500
retrieve 21500:22000
retrieve 22000:22500
retrieve 22500:23000
retrieve 23000:23500
retrieve 23500:24000
retrieve 24000:24500
retrieve 24500:

In [44]:
RA_UP = fetch_entrez_gene_id(list(up_regulated.index.values), input_db='Affy ID')
RA_UP.replace(to_replace='-', value=np.nan, inplace=True)
RA_UP.dropna(how='any',subset=['Gene ID'],inplace=True)
RA_UP['Gene ID'].to_csv(os.path.join(configs.datadir,'RA_UP_GSE21962.txt'), index=False)
RA_UP

INFO    [bioservices:BioDBNet]:  Initialising BioDBNet service (REST)


retrieve 0:76


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


,Gene ID,Ensembl Gene ID
Affy ID,,
204439_at,10964,ENSG00000137959//OTTHUMG00000009724
202388_at,5997,OTTHUMG00000035600//ENSG00000116741
202917_s_at,6279,ENSG00000143546//OTTHUMG00000013124
204959_at,4332,ENSG00000163563//OTTHUMG00000022776
202859_x_at,3576,ENSG00000169429//OTTHUMG00000151316
202973_x_at,10144,ENSG00000138640//OTTHUMG00000161006
229450_at,3437,ENSG00000119917//OTTHUMG00000018708
221731_x_at,1462,OTTHUMG00000131321//ENSG00000038427
201739_at,6446,OTTHUMG00000015613//ENSG00000118515


In [45]:
RA_DOWN = fetch_entrez_gene_id(list(down_regulated.index.values), input_db='Affy ID')
RA_DOWN.replace(to_replace='-', value=np.nan, inplace=True)
RA_DOWN.dropna(how='any',subset=['Gene ID'],inplace=True)
RA_DOWN['Gene ID'].to_csv(os.path.join(configs.datadir,'RA_DOWN_GSE21962.txt'), index=False)
RA_DOWN

INFO    [bioservices:BioDBNet]:  Initialising BioDBNet service (REST)


retrieve 0:81


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


,Gene ID,Ensembl Gene ID
Affy ID,,
210387_at,8970//8339,ENSG00000124635//OTTHUMG00000014446
205898_at,1524,ENSG00000168329//OTTHUMG00000156249
1564139_at,144571,ENSG00000245105//OTTHUMG00000168289
212070_at,9289,OTTHUMG00000176742//ENSG00000205336
215894_at,5729,ENSG00000168229//OTTHUMG00000140299
205495_s_at,10578,ENSG00000115523//OTTHUMG00000130179
230464_at,53637,ENSG00000180739//OTTHUMG00000180581
37145_at,10578,OTTHUMG00000130179//ENSG00000115523
210321_at,2999,ENSG00000100450//OTTHUMG00000140185


In [47]:
# download_gsm_id_maps(configs.datadir, gseXXX, ['GPL570'],vendor='affy')
querytable

,GSE ID,Samples,GPL ID,Instrument
0,GSE21942,GSM1066005,GPL570,affy
1,GSE21942,GSM1066006,GPL570,affy
2,GSE21942,GSM1066007,GPL570,affy
3,GSE21942,GSM1066008,GPL570,affy
4,GSE21942,GSM1066009,GPL570,affy
5,GSE21942,GSM1066010,GPL570,affy
6,GSE21942,GSM1066011,GPL570,affy
7,GSE21942,GSM1066012,GPL570,affy
8,GSE21942,GSM1066013,GPL570,affy
9,GSE21942,GSM1066014,GPL570,affy


In [49]:
data2.to_csv(os.path.join(configs.datadir,'Raw_Fit_GSE21962.csv'), index=True)